In [2]:
import xml.etree.ElementTree as ET
from sklearn import feature_extraction
import sklearn as skl
from nltk.stem import RSLPStemmer
import nltk
from zipfile import ZipFile
import pandas as pd
import numpy as np
import unicodedata
import os
import re
import gensim
from gensim.models import Word2Vec

In [3]:
# Caminho dos arquivos extraidos do kaggle
path_dataset = r'files\classificao-de-notcias.zip'
path_db = r'db'

# Caminho dos arquivos que serão utilizados para a atividade
path_train = r'db\arquivos_competicao\arquivos_competicao\train.csv'
path_test = r'db\arquivos_competicao\arquivos_competicao\test.csv'
path_news = r'db\arquivos_competicao\arquivos_competicao\news'


In [4]:
# Download the stopwords corpus
nltk.download('stopwords')

# Download the RSLPStemmer
nltk.download('rslp')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vitor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\vitor\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

### Extraindo os dados do arquivo .zip, baixado do kaggle

In [5]:
def unzip(path, pathFolder):

    # descompacta a base de dados de notícias
    z = ZipFile(path, 'r')

    if os.path.isdir(pathFolder):
        z.extractall(pathFolder)
        z.close()
    else:
        os.mkdir(pathFolder)
        z.extractall(pathFolder)
        z.close()

    print("Arquivo descompactado com sucesso!")
    
# Antes de descompactar os arquivos valida se ja foram descompactados antes
if not os.path.isdir(path_news):
    unzip(path_dataset, path_db)
else:
    print("Arquivo já descompactado")

Arquivo já descompactado


### Carregando os arquivos de teste e treino

In [6]:
#Carregando os arquivos de treino
df_train = pd.read_csv(path_train)
df_train = df_train.sort_values(['ID'])
df_train.head()

,ID,Class
0,news_00002.xml,Mercados
1,news_00003.xml,Mercados
2,news_00006.xml,Mercados
3,news_00007.xml,Economia
4,news_00008.xml,Mercados


In [7]:
# Carregando os arquivos de teste
df_teste = pd.read_csv(path_test)
df_teste = df_teste.sort_values(['ID'])
df_teste.head()

,ID
0,news_00001.xml
1,news_00004.xml
2,news_00005.xml
3,news_00011.xml
4,news_00015.xml


### Criando um DataFrame com os textos e titulos extraidos do XML
- Para facilitar a aplicação dos metodos foi adicionando novas colunas no df de treino e teste, contem o texto e titulo extraidos dos arquivos xml

In [8]:
def extract_xml_text(path_xml):
    tree = ET.parse(path_xml)
    root = tree.getroot()
    headline = root.find('headline').text if root.find('headline') is not None else ''
    paragraphs = root.findall('.//p')
    text = ' '.join([p.text for p in paragraphs if p.text is not None])

    return headline, text

In [9]:
# Aplicando a função para o df de treino
for idx in df_train.index:
    file = df_train.at[idx, 'ID']
    path_xml = f"{path_news}\{file}"
    titulo, texto = extract_xml_text(path_xml)
    df_train.at[idx, 'TITULO'] = titulo
    df_train.at[idx, 'TEXTO'] = texto

df_train = df_train[['ID', 'TITULO', 'TEXTO', 'Class']]
df_train.head()

,ID,TITULO,TEXTO,Class
0,news_00002.xml,"FUTURO OT DEZ/96 CAI PARA 102,23 CONTRA 102,5...","*Futuro (BDPOTZ6)Dez/96 102,23 vs 102,52 no f...",Mercados
1,news_00003.xml,RESUMO TRANSACÇÕES NO MERCADO CONTÍNUO.,"LISBOA, 30 Set (Reuter) - Transacções no ...",Mercados
2,news_00006.xml,"INDICE PSI20 SOBE 1,28 PONTOS PARA 4.764,93.","LISBOA, 24 Out (Reuter) - O índice PSI20 subi...",Mercados
3,news_00007.xml,ÍNDICE PREÇOS PRODUÇÃO INDUSTRIAL AGO 1996 SO...,"LISBOA, 29 Out (Reuter) - O Índice de Pre...",Economia
4,news_00008.xml,BDP INTERROMPIDA POR PROBLEMAS COMUNICAÇÕES.,"LISBOA, 30 Set (Reuter) - A negociação na Bol...",Mercados


In [10]:
# Aplicando a função para o df de teste
for idx in df_teste.index:
    file = df_teste.at[idx, 'ID']
    path_xml = f"{path_news}\{file}"
    titulo, texto = extract_xml_text(path_xml)
    df_teste.at[idx, 'TITULO'] = titulo
    df_teste.at[idx, 'TEXTO'] = texto

df_teste = df_teste[['ID', 'TITULO', 'TEXTO']]
df_teste.head()

,ID,TITULO,TEXTO
0,news_00001.xml,PROVÁVEL INFLAÇÃO RETOME TENDENCIA DESCENDENT...,LISBOA 12 Set (Reuter) - A inflação deverá re...
1,news_00004.xml,"MMI TRANSACCIONA 178,129 MC, FUTUROS FAZEM 6 ...","LISBOA, 30 Set (Reuter) - O Mercado Monetário..."
2,news_00005.xml,"ACÇÕES SEGUEM POUCO VOLÁTEIS, APATIA DEVE MAN...","LISBOA, 17 Out (Reuter) - As acções do Contín..."
3,news_00011.xml,ESCUDO SEGUE ESTÁVEL E APÁTICO NA MEIA SESSÃO.,O escudo seguia relativamente estável na meia...
4,news_00015.xml,"MMI TRANSACCIONA 234,749 MC, TMP O/N 7,2707 P...","LISBOA, 20 Ago (Reuter) - O Mercado Monetário..."


### Tratando os textos da base de dados
- Aplicada a função de estemização para a linguagem dos textos (português)
- Removendo os ascentos das palavras
- Criando um limite de 200 temrmos por palavras, para evitar que a predição do classificador seja influenciada pelo tamanho da noticia.

In [11]:
def preprocessing_portuguese(text, stemming = False, stopwords = False):
    """
    Funcao usada para tratar textos escritos na lingua portuguesa

    Parametros:
        text: variavel do tipo string que contem o texto que devera ser tratado

        stemming: variavel do tipo boolean que indica se a estemizacao deve ser aplicada ou nao

        stopwords: variavel do tipo boolean que indica se as stopwords devem ser removidas ou nao
    """

    # Lower case
    text = text.lower()

    # remove os acentos das palavras
    nfkd_form = unicodedata.normalize('NFKD', text)
    text = u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

    # remove tags HTML
    regex = re.compile('<[^<>]+>')
    text = re.sub(regex, " ", text)

    # normaliza as URLs
    regex = re.compile('(http|https)://[^\s]*')
    text = re.sub(regex, "<URL>", text)

    # normaliza emails
    regex = re.compile('[^\s]+@[^\s]+')
    text = re.sub(regex, "<EMAIL>", text)

    # converte todos os caracteres não-alfanuméricos em espaço
    regex = re.compile('[^A-Za-z0-9]+')
    text = re.sub(regex, " ", text)

    # normaliza os numeros
    regex = re.compile('[0-9]+.[0-9]+')
    text = re.sub(regex, "NUMERO", text)

    # normaliza os numeros
    regex = re.compile('[0-9]+,[0-9]+')
    text = re.sub(regex, "NUMERO", text)

    # normaliza os numeros
    regex = re.compile('[0-9]+')
    text = re.sub(regex, "NUMERO", text)


    # substitui varios espaçamentos seguidos em um só
    text = ' '.join(text.split())

    # separa o texto em palavras
    words = text.split()

    # trunca o texto para apenas 200 termos
    words = words[0:200]

    # remove stopwords
    if stopwords:
        words = text.split() # separa o texto em palavras
        words = [w for w in words if not w in nltk.corpus.stopwords.words('portuguese')]
        text = " ".join( words )

    # aplica estemização
    if stemming:
        stemmer_method = RSLPStemmer()
        words = [ stemmer_method.stem(w) for w in words ]
        text = " ".join( words )

    # remove palavras compostas por apenas um caracter
    words = text.split() # separa o texto em palavras
    words = [ w for w in words if len(w)>1 ]
    text = " ".join( words )

    return text

In [12]:
# Aplicar a função ao DataFrame de treino
df_train['TEXTO'] = df_train['TEXTO'].apply(preprocessing_portuguese)

# Aplicando a função no df de teste
df_teste['TEXTO'] = df_teste['TEXTO'].apply(preprocessing_portuguese)

### Gerando a representação vetorial

Iremos transformar o texto em um vetor de atributos com valores numéricos. Uma das formas de fazer isso é considerar que cada palavra (ou token) da base de dados de treinamento é um atributo que armazena o número de vezes que uma determinada palavra aparece no texto. Na biblioteca `scikit-learn` podemos fazer essa conversão de texto para um vetor de atributos usando a função `skl.feature_extraction.text.CountVectorizer()`. Essa função gera um modelo de vetorização que pode ser ajustado com a base nos dados de treinamento usando a função `fit_transform()`.


In [13]:
# inicializa o modelo usado para gerar a representação TF (term frequency)
vectorizer = skl.feature_extraction.text.CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None,
                                                         stop_words = None, lowercase = True, binary=False, dtype=np.int32)

# treina o modelo TF com os dados de treinamento e converte os dados de treinamento para uma array que contém a frequência dos termos em cada documento (TF - term frequency)
X_train_tf = vectorizer.fit_transform(df_train['TEXTO'])

# converte os dados de teste
X_test_tf = vectorizer.transform(df_teste['TEXTO'])


print('20 primeiras palavras do vocabulário obtidas a partir dos dados de treinamento:\n')
print(vectorizer.get_feature_names_out()[0:20])

print('\nDimensão dos dados vetorizados: ', X_train_tf.shape)
print('\nDimensão dos dados vetorizados: ', X_test_tf.shape)

20 primeiras palavras do vocabulário obtidas a partir dos dados de treinamento:

['aa' 'aaa' 'aanumero' 'aas' 'abaixo' 'abaixos' 'abaixou' 'abalada'
 'abalaram' 'abanadas' 'abanar' 'abandona' 'abandonado' 'abandonaram'
 'abastecimento' 'abatidas' 'abatimentos' 'abbey' 'abel' 'aberta']

Dimensão dos dados vetorizados:  (2781, 10397)

Dimensão dos dados vetorizados:  (1193, 10397)


In [14]:
X_train_bin = X_train_tf.copy()
X_test_bin = X_test_tf.copy()

#convert os dados para representação binária
X_train_bin[X_train_bin!=0]=1

#convert os dados para representação binária
X_test_bin[X_test_bin!=0]=1 

print(X_train_bin.shape)
print(X_test_bin.shape)

(2781, 10397)
(1193, 10397)


In [15]:
tfidf_model = skl.feature_extraction.text.TfidfTransformer(norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

X_train_tfidf = tfidf_model.fit_transform(X_train_tf)
X_test_tfidf = tfidf_model.transform(X_test_tf)

print(X_train_tfidf.shape)
print(X_test_tfidf.shape)

(2781, 10397)
(1193, 10397)


In [16]:
dataset2_train = []
for i, msg in enumerate(df_train['TEXTO']):
    dataset2_train.append(msg.split())

dataset2_test = []
for i, msg in enumerate(df_teste['TEXTO']):
    dataset2_test.append(msg.split())

print("\n\n20 primeiras palavras da primeira amostra de treino")
print(dataset2_train[0][0:30])

print("\n\n20 primeiras palavras da primeira amostra de teste")
print(dataset2_test[0][0:30])



20 primeiras palavras da primeira amostra de treino
['futuro', 'bdpotzNUMERO', 'dez', 'NUMERO', 'NUMERO', 'vs', 'NUMERO', 'no', 'fecho', 'anterior', 'futuro', 'bono', 'dez', 'NUMERO', 'mffzNUMERO', 'NUMERO', 'vs', 'NUMERO', 'no', 'fecho', 'anterior', 'lisboa', 'editorial', 'NUMERO', 'NUMERO', 'reuters', 'limited', 'NUMERO']


20 primeiras palavras da primeira amostra de teste
['lisboa', 'NUMERO', 'set', 'reuter', 'inflacao', 'devera', 'retomar', 'uma', 'trajectoria', 'descendente', 'logo', 'que', 'cessem', 'os', 'comportamentos', 'anomalos', 'de', 'alguns', 'bens', 'alimentares', 'refere', 'instituto', 'nacional', 'de', 'estatistica', 'ine', 'na', 'sua', 'sintese', 'mensal']


In [17]:
sentencasEmbedding = None
sentencasEmbedding = dataset2_train

embeddingModel = Word2Vec(sentences = sentencasEmbedding,
                          vector_size = 200,
                          window = 3,
                          min_count = 1)

vocabSize = len(embeddingModel.wv)

print("\nTamanho do vocabulário do modelo: ", vocabSize)


Tamanho do vocabulário do modelo:  10398


In [19]:
def getDocvector(model, doc):
    """
    obtem o vetor de cada palavra de um documento e calcula um vetor medio
    """

    wordList = []
    for word in doc:

        try:
            vec = model.wv[word]
            wordList.append(vec)
        except:
            pass

    if len(wordList)>0:
        vetorMedio = np.mean( wordList, axis=0 )
    else:
        vetorMedio = np.zeros( model.wv.vector_size )

    return vetorMedio

In [20]:
def dataset2featureMatrix(dataset, embeddingModel):

    X_embedding = []
    for doc in dataset:
        vec = getDocvector(embeddingModel, doc)
        X_embedding.append(vec)

    X_embedding = np.array(X_embedding)

    return X_embedding

In [21]:
X_train_embedding = dataset2featureMatrix(dataset2_train, embeddingModel)
X_test_embedding = dataset2featureMatrix(dataset2_test, embeddingModel)

print(X_train_embedding.shape)
print(X_test_embedding.shape)

(2781, 200)
(1193, 200)


In [ ]:
import scipy.sparse

# variável que deverá recer o modelo
model = None

# importa o modelo do scikitlearn
import sklearn as skl
from sklearn import linear_model

def classificar(X_train, X_test, Y_train, Y_test):

    ########################## COMPLETE O CÓDIGO AQUI  ########################

    # inicia o classificador
    model = skl.linear_model.LogisticRegression(C=0.5, max_iter = 500,
                                               random_state = 10)

    # normaliza
    if not scipy.sparse.issparse(X_train):
        scaler = skl.preprocessing.StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

    # treina o classificador com os dados de treinameto
    model.fit(X_train, Y_train)

    # treina o classificador com os dados de treinameto
    Y_pred = model.predict(X_test)

    # obtem as medidas de desempenho
    resultados = skl.metrics.classification_report(Y_test, Y_pred)

    print(resultados)

    ##########################################################################

    return model

print("\n\nTreinando com o formato TF")
model_tf = classificar(X_train_tf, X_test_tf, Y_train, Y_test)

print("\n\nTreinando com o formato binário")
model_bin = classificar(X_train_bin, X_test_bin, Y_train, Y_test)

print("\n\nTreinando com o formato TF-IDF")
model_tfidf = classificar(X_train_tfidf, X_test_tfidf, Y_train, Y_test)

print("\n\nTreinando com word embeddings")
model_embedding = classificar(X_train_embedding, X_test_embedding, Y_train, Y_test)
